In [1]:
#import needed package
#---------------------------------------------------------
import pandas as pd
import numpy as np
import sklearn
import autosklearn
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder 
from sklearn.metrics import accuracy_score
import datetime
print('The scikit-learn version is {}.'.format(sklearn.__version__))
import autosklearn.classification
print('The Auto scikit-learn version is {}.'.format(autosklearn.__version__))

The scikit-learn version is 0.19.2.
The Auto scikit-learn version is 0.5.2.


/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:

#Load Data from CSV
#--------------------------------------------------------
filepath1 = '/data/home/eissa/Downloads/datasets_31-benign'
rawdata1 = pd.read_csv( filepath1 ,  sep=','  ,  low_memory=False )
rawdata1['y'] = 0

filepath2 = '/data/home/eissa/Downloads/datasets_31-ack'
rawdata2 = pd.read_csv( filepath2 ,  sep=','  ,  low_memory=False )
rawdata2['y'] = 1

filepath3 = '/data/home/eissa/Downloads/datasets_31-scan'
rawdata3 = pd.read_csv( filepath3 ,  sep=','  ,  low_memory=False )
rawdata3['y'] = 2

filepath4 = '/data/home/eissa/Downloads/datasets_31-syn'
rawdata4 = pd.read_csv( filepath4 ,  sep=','  ,  low_memory=False )
rawdata4['y'] = 3

filepath5 = '/data/home/eissa/Downloads/datasets_31-udp'
rawdata5 = pd.read_csv( filepath5 ,  sep=','  ,  low_memory=False )
rawdata5['y'] = 4

rawdata = pd.concat([rawdata1, rawdata2, rawdata3, rawdata4, rawdata5] , ignore_index=True)

label = 'y'
rawdata.head(10)

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc,y
0,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,...,0.0,0.0,1.000000,60.000000,0.000000,60.000000,0.000000,0.0,0.0,0
1,1.000000,354.000000,0.000000,1.000000,354.000000,0.000000,1.000000,354.000000,0.000000,1.000000,...,0.0,0.0,5.319895,344.262695,4.710446,344.262695,22.188299,0.0,0.0,0
2,1.857879,360.458980,35.789338,1.912127,360.275733,35.923972,1.969807,360.091968,35.991542,1.996939,...,0.0,0.0,6.318264,347.703087,9.034660,347.703087,81.625077,0.0,0.0,0
3,1.000000,337.000000,0.000000,1.000000,337.000000,0.000000,1.000000,337.000000,0.000000,1.000000,...,0.0,0.0,1.000000,337.000000,0.000000,337.000000,0.000000,0.0,0.0,0
4,1.680223,172.140917,18487.448750,1.793580,182.560279,18928.175300,1.925828,193.165753,19153.795810,1.992323,...,0.0,0.0,1.000000,60.000000,0.000000,60.000000,0.000000,0.0,0.0,0
5,2.604076,134.069485,13713.024970,2.744358,142.638274,14811.151900,2.908047,151.843949,15825.289440,2.990476,...,0.0,0.0,1.000000,73.000000,0.000000,73.000000,0.000000,0.0,0.0,0
6,3.103820,119.871020,9719.035896,3.414648,127.222840,11047.423500,3.786586,135.511572,12389.298630,3.977744,...,0.0,0.0,1.000000,90.000000,0.000000,90.000000,0.000000,0.0,0.0,0
7,1.031667,292.532365,1242.373167,1.218057,267.427381,6264.135802,2.513524,200.157230,13785.354110,4.629201,...,0.0,0.0,1.000000,298.000000,0.000000,298.000000,0.000000,0.0,0.0,0
8,1.000003,60.000654,0.155535,1.000558,60.115762,27.494773,1.193809,82.753876,4909.369653,4.582739,...,0.0,0.0,1.000000,60.000000,0.000000,84.852814,0.000000,0.0,0.0,0
9,1.899642,69.475778,80.841721,1.939039,69.339010,93.197642,2.168823,80.561961,2651.375086,5.573056,...,0.0,0.0,1.000000,78.000000,0.000000,78.000000,0.000000,0.0,0.0,0


In [3]:
# split data into Testing and training
# ---------------------------------------------------------------------------------------------------------------------
np.random.seed(1234)
msk = np.random.rand(len(rawdata)) < 0.8
training_data = rawdata[msk]
Testing_data = rawdata[~msk]

X_train = training_data.loc[:, training_data.columns != label]
Y_train = training_data.loc[:, training_data.columns == label]

X_test = Testing_data.loc[:, Testing_data.columns != label]
Y_test = Testing_data.loc[:, Testing_data.columns == label]

In [6]:
import autosklearn.classification
#rawdata
t1 = datetime.datetime.now()
automl = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=100, 
                                                          ensemble_size= 1,
                                                          include_preprocessors=["no_preprocessing"],
                                                          ensemble_memory_limit=45000, 
                                                          seed=1, 
                                                          ml_memory_limit=45000)
automl.fit(X_train, Y_train)
y_hat = automl.predict(X_test)
t2 = datetime.datetime.now()
print("Time:" + str((t2 - t1).total_seconds()))
print("Accuracy score", accuracy_score(Y_test, y_hat))

#automl.cv_results_

#automl.sprint_statistics()

#automl.show_models()


/anaconda/envs/py35/lib/python3.5/site-packages/autosklearn/automl.py:887: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Will change shape via np.ravel().
  y = self._check_y(y)
/anaconda/envs/py35/lib/python3.5/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)


[WARNING] [2020-01-07 14:27:18,977:EnsembleBuilder(1):161390f4c943f6529d8febeacf0d0f54] No models better than random - using Dummy Score!
[WARNING] [2020-01-07 14:27:19,055:EnsembleBuilder(1):161390f4c943f6529d8febeacf0d0f54] No models better than random - using Dummy Score!
Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (93.327893)
[WARNING] [2020-01-07 14:27:21,067:EnsembleBuilder(1):161390f4c943f6529d8febeacf0d0f54] No models better than random - using Dummy Score!
[WARNING] [2020-01-07 14:27:23,079:EnsembleBuilder(1):161390f4c943f6529d8febeacf0d0f54] No models better than random - using Dummy Score!
[WARNING] [2020-01-07 14:27:25,090:EnsembleBuilder(1):161390f4c943f6529d8febeacf0d0f54] No models better than random - using Dummy Score!
[WARNING] [2020-01-07 14:27:27,102:EnsembleBuilder(1):161390f4c943f6529d8febeacf0d0f54] No models better than random - using Dummy Score!
[WARNING] [2020-01-07 14:27:29,1

In [ ]:
accuracy_score(Y_test, y_hat)

In [ ]:
automl.sprint_statistics()